In [ ]:

from ErrP_decoder.data_io import select_eeg_files, load_selected_files

file_paths = select_eeg_files()

datasets = load_selected_files(file_paths)

for ds in datasets:
    print("Data shape:", ds.data.shape)
    print("Info:", ds.info)

In [2]:
from ErrP_decoder.data_io import set_params
for ds in datasets:
    cfg = set_params(ds.info,config_path="../configs/default.yaml")

In [ ]:
from ErrP_decoder.data_io import preprocess_dataset
for ds in datasets:
    ds = preprocess_dataset(ds, cfg)

for ds in datasets:
    neu = sum(ds.trig['typ'] == 0)
    pos = sum(ds.trig['typ'] == 1)
    neg = sum(ds.trig['typ'] == 2)
    print(neu)
    print(pos)
    print(neg)
    


In [ ]:
from ErrP_decoder.data_io import bandpass_filter
from ErrP_decoder.plots import plot_eeg_interactive

for ds in datasets:
    filtered_eeg = bandpass_filter(ds.eeg,cfg)
    plot_eeg_interactive(filtered_eeg, cfg)
    ds.eeg_filt = filtered_eeg

